In [ ]:
from __future__ import absolute_import,division,print_function
import numpy as np
import pandas as pd
import scipy
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn import metrics
from hyperopt import hp, fmin, tpe, hp, STATUS_OK, Trials,rand
from hyperopt.mongoexp import MongoTrials
import h5py
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime
import os
import math
import argparse
import sys
import time
from pyDOE import *
from sklearn.decomposition import PCA
import cv2
from sklearn.metrics import f1_score, accuracy_score  
from tensorflow.contrib.layers import batch_norm, flatten
from tensorflow.contrib.framework import arg_scope
import pickle
import random

use_GPU =False
if use_GPU:
    os.environ['CUDA_VISIBLE_DEVICES']='0'
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
else:    
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
    cpu_num=20
    config = tf.ConfigProto(device_count={"CPU": cpu_num},inter_op_parallelism_threads = cpu_num,
                intra_op_parallelism_threads = cpu_num,log_device_placement=True)

tf.resetf.t_default_graph()
sess = tf.InteractiveSession(config=donfig)
sess.run(tf.group(tf.global_variables_initializer(),tf.local_variables_initializer()))

#load .ckpt type model
'''
#save model as .ckpt type
saver=tf.train.Saver()
saver.save(sess,save_path='*.ckpt')
'''
new_saver = tf.train.import_meta_graph("*.ckpt.meta")
new_saver.restore(sess,".ckpt")

#load .pb type model
'''
#save model as .pb type
builder = tf.saved_model.builder.SavedModelBuilder("path(exclude .pb file name)")
builder.add_meta_graph_and_variables(sess,["tag_string"])
builder.save()
'''
meta_graph_def = tf.saved_model.loader.load(sess, ["tag_string"], "path(exclude .pb file name)")

#get_tensor_by_name and get_operation_by_name
logits = tf.get_collection("logits")
x = sess.graph.get_operation_by_name("input_x").outputs[0]
training_flag = sess.graph.get_operation_by_name("training_flag").outputs[0]

#logits = sess.graph.get_tensor_by_name("finalfully_connected/dense/BiasAdd:0")
#x = sess.graph.operation_by_name("Placeholder").outputs[0]
#training_flag = sess.graph.operation_by_name("Placeholder_2").outputs[0]

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess,coord=coord)

print('####################################')
print (time.strftime("%Y-%m-%d %H:%M:%S",time.localtime()))
te_true_count=0
te_preds=[]
with sess.as_default():
    for i in range(te_labels_1.shape[0]):
        te_1_logits=logits.eval(feed_dict={x: te_imgs_1[i].reshape([-1,image_size,image_size]),training_flag: False})
        te_2_logits=logits.eval(feed_dict={x: te_imgs_2[i].reshape([-1,image_size,image_size]),training_flag: False})
        te_3_logits=logits.eval(feed_dict={x: te_imgs_3[i].reshape([-1,image_size,image_size]),training_flag: False})
        te_4_logits=logits.eval(feed_dict={x: te_imgs_4[i].reshape([-1,image_size,image_size]),training_flag: False})
        te_5_logits=logits.eval(feed_dict={x: te_imgs_5[i].reshape([-1,image_size,image_size]),training_flag: False})

        te_logits =(te_1_logits+te_2_logits+te_3_logits+te_4_logits+te_5_logits)/5
        te_pred = np.argmax(te_logits)
        #print(te_ture)
        te_preds.append(te_pred)
te_acc = accuracy_score(te_preds,te_labels_1)
crosst=pd.crosstab(np.array(te_preds).reshape([-1,]),te_labels_1, margins=True)
print(crosst)
print(te_acc)
coord.request_stop()
coord.join(threads)
sess.close()      